# Metabolome and Lipidome to predict GDM
## Data Exploration

In [ ]:
import os

import pandas as pd
import numpy as np

from sklearn.linear_model import ElasticNet

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ls

## 1. Lipidomics

In [ ]:
xls = pd.ExcelFile('gdm_multiomics_original_datasets/Hurtado_mx_898628_human_plasma_lipidomics_11-2025_submit.xlsx')
print(xls.sheet_names)


In [ ]:
lip_df = pd.read_excel('gdm_multiomics_original_datasets/Hurtado_mx_898628_human_plasma_lipidomics_11-2025_submit.xlsx',sheet_name='Data')


In [ ]:
## split the dataframe into (1)annotation, (2)samples
# annot_df = lip_df.loc[:,:'Unnamed: 6']
annot_df = lip_df.loc[:,'Unnamed: 1':'Unnamed: 2']
## Add 'Annotation' and 'AnnotationSource' columns
annot_df['Annotation']=np.where(~annot_df['Unnamed: 2'].isna(),annot_df['Unnamed: 2'],annot_df['Unnamed: 1'])
annot_df['AnnotationSource']=np.where(~annot_df['Unnamed: 2'].isna(),'annotation','identifier')
annot_df = annot_df[['AnnotationSource','Annotation']]

sample_df = lip_df.loc[:,'MHCMFM1-41':]

## concat annot_df and sample_df
annot_sample_df = pd.concat([annot_df,sample_df],axis=1)

In [ ]:
##select relevant rows & divide them
dx_df = pd.DataFrame(annot_sample_df.loc[3,:]).T
sample_df=annot_sample_df.loc[8:,:]

##concat dianosis + LC-MS samples
dx_sample_df = pd.concat([dx_df,sample_df],axis=0)

## reindexing after the concating
dx_sample_df = dx_sample_df.reset_index().drop(columns='index')

In [ ]:
## Add a columns indication a row type (to distinguish between diagnosis and sample rows)
dx_sample_df["RowType"] = "sample"
dx_sample_df.loc[dx_sample_df.index[0], "RowType"] = "diagnosis"

## Rearrange the columns
dx_sample_df = pd.concat([dx_sample_df.iloc[:,-1],dx_sample_df.iloc[:,:-1]],axis=1)

## Replace a wrong annotation source for diagnosis row
dx_sample_df.loc[0,'AnnotationSource']=np.nan

In [ ]:
dx_sample_df

Summary of lipidomics data:
1. 1st column: annotation of the samples
2. 1st row: GDM diagnosis
3. Annotation column: (1)analyze it only with annotated lipidomics first, (2) then include unannotated ones (from identifier)
4. NAs in the 1st row: Patients without GDM dianosis results - analyze it only with the results

In [ ]:
dx_sample_df.to_csv('gdm_multiomics_cleaned_datasets/gdm_lipidomics_cleaned_df.csv')


## 2. Metabolomics-(1)

In [ ]:
xls = pd.ExcelFile('gdm_multiomics_original_datasets/mx_898521_primary_metabolism.xlsx')
print(xls.sheet_names)

In [ ]:
met_df = pd.read_excel('gdm_multiomics_original_datasets/mx_898521_primary_metabolism.xlsx',sheet_name='sub')

In [ ]:
## split the dataframe into (1)annotation, (2)samples
## the same cleas up process as in lipidomics
##Unnamed: 0 ==BinBase name, Unnamed: 0 == BB id
met_annot_df = met_df[['Unnamed: 0','Unnamed: 3']] 
met_annot_df['Annotation']=np.where(~met_annot_df['Unnamed: 0'].isna(),met_annot_df['Unnamed: 0'],met_annot_df['Unnamed: 3'])
met_annot_df['AnnotationSource']=np.where(~met_annot_df['Unnamed: 0'].isna(),'annotation','identifier')
met_annot_df = met_annot_df[['AnnotationSource','Annotation']]

## extract sample values
met_sample_df = met_df.loc[:,'file id':]
## set column names based on label
met_sample_df.columns = met_sample_df.loc[2]
## remove 'file id' column, and get only sample data
met_sample_df = met_sample_df.iloc[:,1:]

## concat the two dataframes cleaned up (annotation and sample)
met_annot_sample_df = pd.concat([met_annot_df,met_sample_df],axis=1)


In [ ]:
##select relevant rows & divide them
met_dx_df = pd.DataFrame(met_annot_sample_df.loc[6,:]).T
met_sample_df=met_annot_sample_df.loc[8:,:]

##concat dianosis + LC-MS samples
met_dx_sample_df = pd.concat([met_dx_df,met_sample_df],axis=0)

## reindexing after the concating
met_dx_sample_df = met_dx_sample_df.reset_index().drop(columns='index')


In [ ]:
## Add a columns indication a row type (to distinguish between diagnosis and sample rows)
met_dx_sample_df["RowType"] = "sample"
met_dx_sample_df.loc[met_dx_sample_df.index[0], "RowType"] = "diagnosis"

## Rearrange the columns
met_dx_sample_df = pd.concat([met_dx_sample_df.iloc[:,-1],met_dx_sample_df.iloc[:,:-1]],axis=1)

## Replace a wrong annotation source for diagnosis row
met_dx_sample_df.loc[0,'AnnotationSource']=np.nan

met_dx_sample_df

In [ ]:
# ## Testing the 'Annotation column'
# met_dx_sample_df.head(135) ## annotation (name) disappears from row 130, and substituted with numbers
# type(met_dx_sample_df['Annotation'][135]) ## check the type of a number => integer
# met_dx_sample_df['Annotation'].apply(type).unique() ## there are str, integer, float in the column

# ## checking whether I can separate 'annotation' and 'identifier' using datatype
# met_dx_sample_df['Annotation'].apply(lambda x: isinstance(x, (float, int)))
# met_dx_sample_df[
#     met_dx_sample_df['Annotation'].apply(lambda x: isinstance(x, (float, int)))
# ] ## 323 rows

# met_dx_sample_df[
#     met_dx_sample_df['Annotation'].apply(lambda x: isinstance(x, (str)))
# ]  ##128 rows
# met_dx_sample_df # 451 rows 

"""
conclusion: in this file, annotation name is written as str. When there is no annotation name, a number (either int or float) is recorded in the same row.
Fortunatley, all annotations are written in str type while all identifiers are written in int/float type is this file.
We separated these twe types using data type.
"""

## If data type is either int or float from 'Annotation column', assign 'identifier' in 'AnnotationSource' column
mask = met_dx_sample_df['Annotation'].apply(
    lambda x: isinstance(x, (int, float))
)

met_dx_sample_df.loc[mask, 'AnnotationSource'] = 'identifier'


In [ ]:
met_dx_sample_df

Questions regarding metabolomic data (mainly differences between the lipidomics and metabolomics):
1. I used 'label' as an identifier (I guess each column indicates one patient), but it seems like the labels between lipidomics and metabolomics doesn't match??
    - Are they the same patients?
    - Should I match the patients between three files (2 metabolomics + 1 lipidomics)
2. In lipidomics, names of those without labelings were 'null', so I used 'identifier numbers' to identify unique lipidomics. However, in metabolomics, metabolomics without names still have "identifier numbers" in BinBase name, so I used it, but it seems like it doesn't match with BB id.

In [ ]:
met_dx_sample_df.to_csv('gdm_multiomics_cleaned_datasets/gdm_metabolomics_1_cleaned_df.csv')

## (3) Metabolomics-(2)

In [ ]:
xls = pd.ExcelFile('gdm_multiomics_original_datasets/mx_898735_Hurtado_HILIC_human_plasma_01-2026_submit.xlsx')
print(xls.sheet_names)

In [ ]:
met2_df = pd.read_excel('gdm_multiomics_original_datasets/mx_898735_Hurtado_HILIC_human_plasma_01-2026_submit.xlsx',sheet_name='submit')


In [ ]:
met2_df.head(10)

In [ ]:
met2_df.iloc[:10,:20]

In [ ]:
### Test how identifier can be distinguished in this file
# met2_df.iloc[:,:2]

# met2_df['SourceTest']= (met2_df['Unnamed: 0']==met2_df['Unnamed: 1'])
# met2_df[['Unnamed: 0','Unnamed: 1','SourceTest']].head(403)

"""
conclusion: input of Unnamed: 1 (annotation name) column is equal with that of Unnamed:0 (identifier) column when there is no annotation name
Test whether inputs in the two columns are equal in each row, and assign 'annotation' if they are different while assgin 'identifier' if they are the same
"""

In [ ]:
met2_df['AnnotationSource'] = np.where(met2_df['Unnamed: 0']==met2_df['Unnamed: 1'],'identifier','annotation')

In [ ]:
met2_annot_df = met2_df[['AnnotationSource','Unnamed: 1']]

met2_sample_df = met2_df.loc[:,'Label':]
met2_sample_df = met2_sample_df.iloc[:,1:]

# concat the two dataframes cleaned up (annotation and sample)
met2_annot_sample_df = pd.concat([met2_annot_df,met2_sample_df],axis=1)


In [ ]:
# Rename the annotation column
met2_annot_sample_df = met2_annot_sample_df.rename(columns={
    "Unnamed: 1": "Annotation"
})
met2_annot_sample_df.head(10)

In [ ]:
##select relevant rows & divide them
met2_dx_df = pd.DataFrame(met2_annot_sample_df.loc[3,:]).T
met2_sample_df=met2_annot_sample_df.loc[9:,:]

##concat dianosis + LC-MS samples
met2_dx_sample_df = pd.concat([met2_dx_df,met2_sample_df],axis=0)

## reindexing after the concating
met2_dx_sample_df = met2_dx_sample_df.reset_index().drop(columns='index')


In [ ]:
met2_dx_sample_df

In [ ]:
## Replace a wrong annotation source for diagnosis row
met2_dx_sample_df.loc[0,'AnnotationSource']=np.nan

## Add a columns indication a row type (to distinguish between diagnosis and sample rows)
met2_dx_sample_df["RowType"] = "sample"
met2_dx_sample_df.loc[met2_dx_sample_df.index[0], "RowType"] = "diagnosis"

## Rearrange the columns
met2_dx_sample_df = pd.concat([met2_dx_sample_df.iloc[:,-1],met2_dx_sample_df.iloc[:,:-1]],axis=1)

met2_dx_sample_df

Questions regarding annotation
- For metabolomics, I may add "AnnotationSource" manually, as it has id automatically when 'annotation name' is unavailable

In [ ]:
met2_dx_sample_df.to_csv('gdm_multiomics_cleaned_datasets/gdm_metabolomics_2_cleaned_df.csv')